# Project
#### This project is to get the windfarm data from accross ireland
#### Windfarm data is getting from 
##### https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2025.xlsx
##### https://cms.eirgrid.ie/sites/default/files/2025-02/System_Data_Qtr_Hourly_2024.xlsx
##### https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2022-2023_0.xlsx
##### https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2020-2021.xlsx
##### https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2018-2019.xlsx
##### https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2016-2017.xlsx
##### https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2014-2015.xlsx

In [25]:
import sys
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [19]:
# Get data from excel files from EirGrid website
df_2025 = pd.read_excel('https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2025-v10.xlsx', index_col=0)
df_2024 = pd.read_excel('https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2024.xlsx', index_col=0)
df_2022 = pd.read_excel('https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Quarter-Hourly-2022-2023.xlsx', index_col=0)
df_2020 = pd.read_excel('https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2020-2021.xlsx', index_col=0)
df_2018 = pd.read_excel('https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2018-2019.xlsx', index_col=0)
df_2016 = pd.read_excel('https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2016-2017.xlsx', index_col=0)
df_2014 = pd.read_excel('https://cms.eirgrid.ie/sites/default/files/publications/System-Data-Qtr-Hourly-2014-2015.xlsx', index_col=0)

NameError: name 'pd' is not defined

In [ ]:
# Removed unnamed column from data frame
df_2022
df_2025 = df_2025.loc[:, ~df_2025.columns.str.contains('^Unnamed')]
df_2024 = df_2024.loc[:, ~df_2024.columns.str.contains('^Unnamed')]
df_2022 = df_2022.loc[:, ~df_2022.columns.str.contains('^Unnamed')]
df_2020 = df_2020.loc[:, ~df_2020.columns.str.contains('^Unnamed')]
df_2018 = df_2018.loc[:, ~df_2018.columns.str.contains('^Unnamed')]
df_2016 = df_2016.loc[:, ~df_2016.columns.str.contains('^Unnamed')]
df_2014 = df_2014.loc[:, ~df_2014.columns.str.contains('^Unnamed')]


In [ ]:
# Merge all data in one data frame
df = pd.concat([df_2014, df_2016, df_2018, df_2020, df_2022, df_2024, df_2025])
df.head(5)

In [ ]:
# Get list of columns in data frame to removed unwanted columns
df.columns.to_list()

In [ ]:
# removed unwanted columns that are not needed
df = df.drop(columns=['GMT Offset', 'NI Generation', 'NI Demand', 'NI Wind Availability', 'NI Wind Generation','SNSP', 'NI: Northern Ireland\nIE: Ireland', 'NI Solar Availability', 'NI Solar Generation', 'Moyle I/C', 'NI Wind Penetration', 'NI Solar Penetration','EWIC I/C','AI Generation', 'AI Demand', 'AI Wind Availability', 'AI Wind Generation', 'AI Solar Availability', 'AI Solar Generation', 'AI Hydro', 'Inter-Jurisdictional Flow', 'AI Wind Penetration', 'AI Solar Penetration', 'AI Oversupply', 'AI Oversupply Percentage', 'NI: Northern Ireland \nIE: Ireland \nAI: All Island ','For queries relating this report, please contact: RenewableReports@EirGrid.com\nNI: Northern Ireland \nIE: Ireland \nAI: All Island '])
df.head(5)

In [ ]:
# Convert NaN values to 0
df = df.fillna(0)
df.head(5)

In [ ]:
# calculate the energy generation againest demand to get surplus/deficit
df['energysurplus'] = df['IE Generation'] - df['IE Demand']
df.head(5)

In [ ]:
# get the year from the index and add it as a new column
df['year'] = df.index.year
df.head(5)

In [ ]:
# plot the bar chart of energy surplus year wise
# get the sum of energy surplus for each year using groupby function
yearly_surplus = df.groupby('year')['energysurplus'].sum()
# plot the bar chart
plt.figure(figsize=(30,10))
yearly_surplus.plot(kind='bar', color='skyblue')
plt.title('Yearly Energy Surplus / Deficit')
plt.xlabel('year')
plt.ylabel('Energy Surplus (MWh)')
plt.gcf().autofmt_xdate(rotation=45)

In [ ]:
# Get the data for the year and month wise
# Get year and month from the date index
df['month'] =  df.index.year.astype(dtype='str') + "-" + df.index.strftime('%m').astype(dtype='str')
df.head(5)

In [ ]:
# plot the bar chart of energy surplus Month Year wise
# get the sum of energy surplus for each Month year using groupby function
monthly_surplus = df.groupby('month')['energysurplus'].sum()
# plot the bar chart
plt.figure(figsize=(30,10))
monthly_surplus.plot(kind='bar', color='skyblue')
plt.title('Monthly Energy Surplus / Deficit')
plt.xlabel('Month-Year')
plt.ylabel('Energy Surplus (MWh)')
plt.gcf().autofmt_xdate(rotation=45)
#plt.tight_layout()

In [ ]:
# get energy generation from other sources
df['IE Other Generation'] = df['IE Generation'] - (df['IE Wind Generation'] + df['IE Solar Generation'] + df['IE Hydro'])
df.head(5)

In [ ]:
# create a pie chart of energy source contribution to total generation
yearly_generation_type = df.groupby(df['year'])[['IE Wind Generation','IE Solar Generation','IE Hydro','IE Other Generation']].sum()
yearly_generation_type


In [ ]:

yearly_generation_type.loc[2014].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2014')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2015].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2015')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2016].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2016')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2017].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2017')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2018].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2018')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2019].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2019')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2020].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2020')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2021].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2021')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2022].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2022')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2023].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2023')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2024].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2024')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()
yearly_generation_type.loc[2025].plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Energy Source Contribution to Total Generation in 2025')
plt.ylabel('')
plt.autoscale(enable=True,tight=True)
plt.show()